In [16]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import os

In [2]:
# change the directiory to root to allow importing of py files saved in algorithmictrading
os.chdir(r'..\\')
from algorithmictrading.secrets import IEX_CLOUD_API_TOKEN

# Data Loading - S&P 500 Index
The S&P 500 Index is one of the most common benchmarks for US Large Cap stocks. It tracks the performance of 500 of the largest companies in the United States.

You can substitute any list of tickers for this equal weight walk-through. The list of stocks should be aved in the *data* folder.

In [6]:
stocks = pd.read_csv(r'.\data\sp_500_stocks.csv')

In [7]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


# Connecting to the IEX API
We will be using the free IEX Cloud API for the market data. Prices are purpsefully scrambled and are NOT meant for production!

[Documentation can be found here.](https://iexcloud.io/docs/api/#testing-sandbox)

We can use the base URL and pass `symbol` and `token` to connect to the API and pull in the correct data.

In [291]:
BASE_URL = 'https://sandbox.iexapis.com/stable'
symbol = 'AAPL'
quote = f'/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

In [292]:
data = requests.get(BASE_URL + quote).json()

data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'LAAEEO(CE GAA)R TN/S GLQSMKTSDBNL',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'iflfiaoc',
 'close': None,
 'closeTime': None,
 'closeSource': 'ciafflio',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 131.21,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'January 12, 2021',
 'latestUpdate': 1651745548692,
 'latestVolume': None,
 'iexRealtimePrice': 134.36,
 'iexRealtimeSize': 938,
 'iexLastUpdated': 1621590519543,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 129.63,
 'previousVolume': 103997884,
 'change': -0.1,
 'changePercent': -0.00078,
 'volume': None,
 'iexMarketPercent': 0.009192691546355637,
 'iexVo

The data is stored as a `dict`, which means we can easily access the attribute by specifying it's key.

In [293]:
data['companyName']

'Apple Inc'

In order to get stock information, we can apply a function across each row in the `dataframe`. A helper function has been created to all a symbol and a specified field to be passed as arguments.

In [57]:
def get_api_data(symbol, field):
    quote = f'/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(BASE_URL + quote).json()
    return data[field]

In [60]:
df = stocks.copy()
df = (df.assign(stock_price= lambda x: x['Ticker'].apply(get_api_data, field='latestPrice'),
                market_cap= lambda x: x['Ticker'].apply(get_api_data, field='marketCap')))

In [61]:
df.head()

,Ticker,stock_price,market_cap
0,A,132.610,39.791959
1,AAL,15.247,9.337053
2,AAP,182.520,12.002677
3,AAPL,131.639,2232.828754
4,ABBV,111.054,201.947629


# Making Batch API Calls
Using `.apply()` works - however it is *really* slow. We can significantly speed up the data retrieval process by making batch requests through the API instead of making 500+ http requests (one for each ticker). The IEX API has a limit of 100 tickers per batch request. 

In [270]:
def make_chunks(df):
     return np.array_split(df['Ticker'].to_list(), np.ceil(len(df) / 100))

def get_data_batch(df):
    df_list = []
    chunks = make_chunks(df)
    for chunk in chunks:
        ticker_strings = ','.join(chunk)
        batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={ticker_strings}&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(batch_api_call_url).json()
        tickers = [k for k in data.keys()]
        latestprices = [data[k]['quote']['latestPrice'] for k in data.keys()]
        marketcaps = [data[k]['quote']['marketCap'] for k in data.keys()]
        df = pd.DataFrame({'ticker': tickers, 'latest_price': latestprices, 'market_cap': marketcaps})
        df_list.append(df)
    return  pd.concat(df_list, ignore_index=True)


In [294]:
batch_df = get_data_batch(stocks)

In [295]:
batch_df.head()

,ticker,latest_price,market_cap
0,A,131.79,40857084296
1,AAL,15.42,9459478008
2,AAP,182.84,12074804386
3,AAPL,132.90,2225387810895
4,ABBV,112.08,196686690146


# Calculating Share Amounts
In order to calculate the share amounts needed for each ticker, we will beed the size of the portfolio. 

The below function evenly divides the portfolio dollar amount amongst each name in the index, and calculates the amount of shares based on that number.

Note that we are rounding down partial share amounts to the nearest whole number so we don't go over the amount we wish to invest.

In [296]:
def get_share_amounts(df, portfolio_size=50000000):
    share_amounts = portfolio_size / len(df.index)
    return df.assign(recommended_trades= lambda x: np.floor(share_amounts /  x['latest_price']))

In [297]:
final_df = get_share_amounts(batch_df, 40000000)

final_df.head()

,ticker,latest_price,market_cap,recommended_trades
0,A,131.79,40857084296,601.0
1,AAL,15.42,9459478008,5136.0
2,AAP,182.84,12074804386,433.0
3,AAPL,132.90,2225387810895,595.0
4,ABBV,112.08,196686690146,706.0


# Exporting Data to Excel

Pandas can easily output to a csv file of xlsx file natively. However, if we want to output to a styled xlsx file, we can use `xlsxwriter` to customize the output to a much greater degree. 

In [301]:
writer = pd.ExcelWriter(r'.\data\equal_weight_recommended_trades.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

dollar_format = writer.book.add_format(
        {
        'num_format':'$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

integer_format = writer.book.add_format(
        {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

market_cap_format = writer.book.add_format(
        {
        'num_format':'$#,##0.0,,', # millions
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1,
        'border_color': font_color
        }
    )

column_formats = { 
                    'A': ['ticker', string_format],
                    'B': ['latest_price', dollar_format],
                    'C': ['market_cap', market_cap_format],
                    'D': ['recommended_trades', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)
    
writer.sheets['Recommended Trades'].hide_gridlines(2)

writer.save()